In [ ]:
bucket = "tabbie_financial"

In [ ]:
import pandas as pd

In [7]:
data = pd.read_csv("gs://{}/data/consumer_complaints.csv".format(bucket))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [13]:
import os
import numpy as np
import pandas as pd
#%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import LabelBinarizer

print(tf.__version__)


1.15.2


In [14]:
data = data[['product', 'consumer_complaint_narrative']]

In [18]:
def preprocess(data):
    import re
    data = data[pd.notnull(data['consumer_complaint_narrative'])]
    pd.set_option('max_colwidth', 1000)
    data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.lower()
    chr = ['x', '{', '}', '/']
    data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('x', '')
    data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('{', '')
    data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('}', '')
    data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('/', '')
    
    return data
    

In [19]:
data = preprocess(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [7]:
data = data[pd.notnull(data['consumer_complaint_narrative'])]
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
190126,03/19/2015,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,XXXX has claimed I owe them {$27.00} for XXXX ...,NaN,"Diversified Consultants, Inc.",NY,121XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290516
190135,03/19/2015,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,Due to inconsistencies in the amount owed that...,NaN,M&T Bank Corporation,VA,221XX,Servicemember,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290492
190155,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,In XX/XX/XXXX my wages that I earned at my job...,NaN,Wells Fargo & Company,CA,946XX,NaN,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290524
190207,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,I have an open and current mortgage with Chase...,NaN,JPMorgan Chase & Co.,CA,900XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290253
190208,03/19/2015,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,NaN,XXXX was submitted XX/XX/XXXX. At the time I s...,NaN,Rushmore Loan Management Services LLC,CA,956XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1292137


In [8]:
pd.set_option('max_colwidth', 1000)
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.lower()
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
190126,03/19/2015,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,xxxx has claimed i owe them {$27.00} for xxxx years despite the proof of payment i sent them : canceled check and their ownpaid invoice for {$27.00}! \nthey continue to insist i owe them and collection agencies are after me. \nhow can i stop this harassment for a bill i already paid four years ago? \n,NaN,"Diversified Consultants, Inc.",NY,121XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290516
190135,03/19/2015,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,"due to inconsistencies in the amount owed that i was told by m & t bank and the amount that was reported to the credit reporting agencies, i was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. i had a vehicle that was stolen and it was declared a total loss by insurance company. the insurance company and the gap insurancw companypaid the outstanding balance of the loan, but i was told by m & t bank that there was still a balance due on the loan. in good faith, without having received any proof as to why there was still a balance, i made a partial payment towards the remaining debt. i then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. at one point, in xxxx 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when i checked my credit report again on xxxx x...",NaN,M&T Bank Corporation,VA,221XX,Servicemember,Consent provided,Web,03/19/2015,Closed with explanation,Yes,No,1290492
190155,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,"in xx/xx/xxxx my wages that i earned at my job decreased by almost half, by xx/xx/xxxx i knew i was in trouble with my home loan. i began contacting wfb whom my home loan is with, for assitance and options. \nin early xx/xx/xxxx i began the loan modification process with wells fargo bank. i was told that they would not assist me with anything financial on my home loan until i fell 90 days behind, though at the time i started to inquire for assistance from wfb i was only a few weeks behind. so, i began working with a program called xxxx. they approved me for a variety of assistence and reached out to wells fargo bank to determine what they could assist with. wells fargo then turned down the assistance from xxxx and finally offered to do a loan modification for me. the outcome was totally unknow about what i would be offered in the end by wfb for assistance. wells fargo lost my paperwork twice during this process, so it took 2 months from the time i started to the time my paperwork b...",NaN,Wells Fargo & Company,CA,946XX,NaN,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290524
190207,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,"i have an open and current mortgage with chase bank # xxxx. chase is reporting the loan payments to xxxx but xxxx is surpressing the information and reporting the loan as discharged in bk. this mortgage was reaffirmed in a chapter xxxx bk discharged dated xxxx/xxxx/2013. chase keeps referring to bk law for chapter xxxx and we keep providing documentation for chapter xxxx, and the account should be open and current with all the payments \n",NaN,JPMorgan Chase & Co.,CA,900XX,Older American,Consent provided,Web,03/19/2015,Closed with explanation,Yes,Yes,1290253
190208,03/19/2015,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,NaN,"

In [9]:
data = data[['product', 'consumer_complaint_narrative']]

In [10]:
import re
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('x', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('{', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('}', '')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('/', '')
data.head()

,product,consumer_complaint_narrative
190126,Debt collection,has claimed i owe them $27.00 for years despite the proof of payment i sent them : canceled check and their ownpaid invoice for $27.00! \nthey continue to insist i owe them and collection agencies are after me. \nhow can i stop this harassment for a bill i already paid four years ago? \n
190135,Consumer Loan,"due to inconsistencies in the amount owed that i was told by m & t bank and the amount that was reported to the credit reporting agencies, i was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. i had a vehicle that was stolen and it was declared a total loss by insurance company. the insurance company and the gap insurancw companypaid the outstanding balance of the loan, but i was told by m & t bank that there was still a balance due on the loan. in good faith, without having received any proof as to why there was still a balance, i made a partial payment towards the remaining debt. i then sent the goodwill letter still offering to pay the remainder of the debt, but in echange for the removal of the negative entry on my credit report. at one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of $0.00, but when i checked my credit report again on 2015, there..."
190155,Mortgage,"in my wages that i earned at my job decreased by almost half, by i knew i was in trouble with my home loan. i began contacting wfb whom my home loan is with, for assitance and options. \nin early i began the loan modification process with wells fargo bank. i was told that they would not assist me with anything financial on my home loan until i fell 90 days behind, though at the time i started to inquire for assistance from wfb i was only a few weeks behind. so, i began working with a program called . they approved me for a variety of assistence and reached out to wells fargo bank to determine what they could assist with. wells fargo then turned down the assistance from and finally offered to do a loan modification for me. the outcome was totally unknow about what i would be offered in the end by wfb for assistance. wells fargo lost my paperwork twice during this process, so it took 2 months from the time i started to the time my paperwork began to be processed for some kind of ..."
190207,Mortgage,"i have an open and current mortgage with chase bank # . chase is reporting the loan payments to but is surpressing the information and reporting the loan as discharged in bk. this mortgage was reaffirmed in a chapter bk discharged dated 2013. chase keeps referring to bk law for chapter and we keep providing documentation for chapter , and the account should be open and current with all the payments \n"
190208,Mortgage,"was submitted . at the time i submitted this complaint, i had dealt with rushmore mortgage directly endeavoring to get them to stop the continuous daily calls i was receiving trying to collect on a mortgage for which i was not responsible due to bankruptcy. they denied having knowledge of the bankruptcy, even though i had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. when they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in federal court. on rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in . i would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they are going to remove it from all my b..."


In [11]:
data['product'].unique()

array(['Debt collection', 'Consumer Loan', 'Mortgage', 'Credit card',
       'Credit reporting', 'Student loan', 'Bank account or service',
       'Payday loan', 'Money transfers', 'Other financial service',
       'Prepaid card'], dtype=object)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66806 entries, 190126 to 553096
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   product                       66806 non-null  object
 1   consumer_complaint_narrative  66806 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [20]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66806 entries, 190126 to 553096
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   product                       66806 non-null  object
 1   consumer_complaint_narrative  66806 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [21]:
data_comp=data[['consumer_complaint_narrative']]
data_prod=data[['product']]

In [22]:
train_size = int(len(data_comp) * .999)

train_descriptions = data_comp[:train_size].astype('str')
train_prod = data_prod[:train_size]

test_descriptions = data_comp[train_size:].astype('str')
test_prod =data_prod[train_size:]

In [23]:
print(train_descriptions.shape)
print(test_descriptions.shape)

(66739, 1)
(67, 1)


In [24]:
train_size = int(len(train_descriptions) * .8)

train_desc = train_descriptions[:train_size]
train_pr = train_prod[:train_size]

val_desc = train_descriptions[train_size:]
val_pr =train_prod[train_size:]

In [27]:
print(train_desc.shape)
print(val_desc.shape)

(53391, 1)
(13348, 1)


### Install Kubeflow pipelines SDK

In [29]:
!pip3 install kfp --upgrade --user

Requirement already up-to-date: kfp in /home/jovyan/.local/lib/python3.6/site-packages (1.0.1)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Check if the install was successful:

In [30]:
!which dsl-compile

## Build the Components

In [31]:
# Import Kubeflow SDK
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [33]:
out_dir = "/home/jovyan/11-financial-products/data/out/"

In [37]:
data.to_csv("data/input", index=False)

In [43]:
in_dir = "/home/jovyan/11-financial-products/data/input"

In [ ]:
def train(in_data, out_data, model_file):
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    import tensorflow_hub as hub
    import json
    import urllib
    
    from sklearn.preprocessing import LabelBinarizer
    from sklearn.model_selection import train_test_split
    
    input_ = pd.read_csv(in_data)
    # split the data
    X = input_['consumer_complaint_narrative'].astype('str')
    y = input_['product']
    X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
    
    encoder = LabelBinarizer()
    encoder.fit_transform(y_train)
    train_encoded = encoder.transform(y_train)
    val_encoded = encoder.transform(y_test)
    num_classes = len(encoder.classes_)
    
    description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3", trainable=False)
    
    multi_label_head = tf.contrib.estimator.multi_label_head(
        num_classes,
        loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
    )
    
    features = {
        "descriptions": np.array(X_train).astype(np.str)
    }
    labels = np.array(train_encoded).astype(np.int32)
    train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)

    estimator = tf.estimator.DNNEstimator(
        head=multi_label_head,
        hidden_units=[64,10],
        feature_columns=[description_embeddings])
    
    estimator.train(input_fn=train_input_fn)
    
    # Define our eval input_fn and run eval
    eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(X_test).astype(np.str)}, val_encoded.astype(np.int32), shuffle=False)
    estimator.evaluate(input_fn=eval_input_fn)
    
    #Save the model to the designated
    # estimator.save(f'{out_data}/{model_file}')
    serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
          tf.feature_column.make_parse_example_spec(feature_columns))
    estimator.export_saved_model(f'{out_data}/{model_file}', serving_input_fn)
    
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{out_data}/test_data', 'wb') as f:
        pickle.dump((X_test,  y_test), f)


In [ ]:
classifier = train(in_dir, out_dir, "model")

In [ ]:
#loading saved model
imported = tf.saved_model.load(out_dir + "/model")

In [22]:
print(train_pr.shape)
print(val_pr.shape)

(53391, 1)
(13348, 1)


In [23]:
print(train_desc.info())
print(val_desc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53391 entries, 190126 to 503087
Data columns (total 1 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   consumer_complaint_narrative  53391 non-null  object
dtypes: object(1)
memory usage: 834.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13348 entries, 503088 to 552770
Data columns (total 1 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   consumer_complaint_narrative  13348 non-null  object
dtypes: object(1)
memory usage: 208.6+ KB
None


In [24]:
from sklearn import preprocessing
encoder = preprocessing.LabelBinarizer()
encoder.fit_transform(train_pr)
train_encoded = encoder.transform(train_pr)
val_encoded = encoder.transform(val_pr)
num_classes = len(encoder.classes_)

# Print all possible products and the label for the first complaint in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Bank account or service' 'Consumer Loan' 'Credit card'
 'Credit reporting' 'Debt collection' 'Money transfers' 'Mortgage'
 'Other financial service' 'Payday loan' 'Prepaid card' 'Student loan']
[0 0 0 0 1 0 0 0 0 0 0]


In [25]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3", trainable=False)

In [26]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [27]:
features = {
  "descriptions": np.array(train_desc).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)

estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp663xez2q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c3bf59ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp663xez2q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c3bf59ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
features = {
  "descriptions": np.array(train_desc).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)

estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp68d3h7da', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c41c71d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp68d3h7da', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9c41c71d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
%%timeit
estimator.train(input_fn=train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:Saving checkpoints for 21360 into /tmp/tmp68d3h7da/model.ckpt.


INFO:tensorflow:Saving checkpoints for 21360 into /tmp/tmp68d3h7da/model.ckpt.


INFO:tensorflow:Loss for final step: 0.054397263.


INFO:tensorflow:Loss for final step: 0.054397263.


1 loop, best of 3: 17min 25s per loop


In [30]:
%%timeit
train_input_fn_1 = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(train_desc).astype(np.str)}, train_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=train_input_fn_1)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 21360: auc = 0.9627397, auc_precision_recall = 0.8144742, average_loss = 0.11937788, global_step = 21360, loss = 0.11926943


INFO:tensorflow:Saving dict for global step 21360: auc = 0.9627397, auc_precision_recall = 0.8144742, average_loss = 0.11937788, global_step = 21360, loss = 0.11926943


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21360: /tmp/tmp68d3h7da/model.ckpt-21360


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21360: /tmp/tmp68d3h7da/model.ckpt-21360


1 loop, best of 3: 1min 45s per loop


In [31]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(val_desc).astype(np.str)}, val_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 21360: auc = 0.95120925, auc_precision_recall = 0.782696, average_loss = 0.13244736, global_step = 21360, loss = 0.1325346


INFO:tensorflow:Saving dict for global step 21360: auc = 0.95120925, auc_precision_recall = 0.782696, average_loss = 0.13244736, global_step = 21360, loss = 0.1325346


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21360: /tmp/tmp68d3h7da/model.ckpt-21360


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21360: /tmp/tmp68d3h7da/model.ckpt-21360


{'auc': 0.95120925,
 'auc_precision_recall': 0.782696,
 'average_loss': 0.13244736,
 'global_step': 21360,
 'loss': 0.1325346}

In [32]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, shuffle=False)

results = estimator.predict(predict_input_fn)

In [33]:
# Display predictions
for product in results:
  top = product['probabilities'].argsort()[-1:]
  for prod in top:
    text_prod = encoder.classes_[prod]
    print(text_prod + ': ' + str(round(product['probabilities'][prod] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Debt collection: 33.14%

Debt collection: 70.93%

Debt collection: 72.25%

Credit reporting: 86.28%

Debt collection: 83.93%

Debt collection: 74.52%

Credit reporting: 29.01%

Mortgage: 87.63%

Mortgage: 99.7%

Debt collection: 53.34%

Debt collection: 94.84%

Credit reporting: 95.39%

Credit card: 96.93%

Debt collection: 69.18%

Debt collection: 90.7%

Mortgage: 38.23%

Credit reporting: 97.77%

Mortgage: 79.43%

Credit card: 39.63%

Bank account or service: 72.2%

Student loan: 52.07%

Mortgage: 17.59%

Mortgage: 93.27%

Credit reporting: 64.09%

Credit card: 21.52%

Debt collection: 45.24%

Mortgage: 91.84%

Debt collection: 42.54%

Mortgage: 99.42%

Mortgage: 40.68%

Credit card: 44.89%

Mortgage: 82.66%

Credit card: 55.47%

Mortgage: 98.82%

Debt collection: 48.24%

Debt collection: 72.43%

Bank account or service: 84.71%

Mortgage: 83.86%

Credit reporting: 99.24%

Mortgage: 65.69%

Credit card: 65.96%

Credit reporting: 99.04%

Mortgage: 96.08%

Mortgage: 96.67%

Credit card:

In [34]:
test_prod

,product
552772,Credit card
552773,Debt collection
552775,Debt collection
552779,Credit reporting
552792,Debt collection
...,...
553084,Credit reporting
553085,Credit reporting
553086,Payday loan
553090,Mortgage
